In [1]:
import spotipy
import spotipy.util as util
import pickle
import sys
import requests
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from multiprocessing import Pool
from fuzzywuzzy import fuzz
import string
import json
import unidecode

/Users/iliasmiraoui/anaconda3/lib/python3.7/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:


# Spotify Infos
scope = 'user-library-read'
username = "ilias.miraoui@gmail.com"

token = util.prompt_for_user_token(username, scope, client_id='6d6b00a4a500471e94c5a3f1559e8cc6', client_secret='500950b79db94ac88f6859d24e4e6033', redirect_uri='http://localhost:8888/callback')
if token:
    spotify = spotipy.Spotify(auth=token)
else:
    print("Connexion to spotify API failed.")
    sys.exit()

    


In [3]:
def simplify_string(s):
    return unidecode.unidecode(s.lower()).replace('the', ' ').replace(' ', '').translate(str.maketrans('', '', string.punctuation))


def similar(a, b, threshold=80):
    return fuzz.ratio(simplify_string(a), simplify_string(b)) >= threshold

In [10]:
def get_song_details(artist,track):
    song_details = dict()
    results = spotify.search(q=" track:" + track + ' artist:' + artist  , type='track')
    if results['tracks']['total'] > 0:
        for i in results['tracks']['items']:
            if similar(track, i['name'], 80) and similar(artist, i["artists"][0]["name"], 80):
                maxi = i
                break
        if maxi is None:
            return None
        if results is None or len(results) == 0:
            return None
    _id = maxi["id"] 
    print(maxi)
    song_details[_id] = dict()
    song_details[_id]["explicit"] = maxi["explicit"]
    song_details[_id]["duration_ms"] = maxi["duration_ms"]
    song_details[_id]["disc_number"] = maxi["disc_number"]
    song_details[_id]["track_number"] = maxi["track_number"]
    return song_details


    

In [11]:
def get_song_features(_id):
    song_features = dict()
    results = spotify.audio_features(_id)[0]
    song_features["danceability"] = results["danceability"]
    song_features["energy"] = results["energy"]
    song_features["key"] = results["key"]
    song_features["loudness"] = results["loudness"]
    song_features["mode"] = results["mode"]
    song_features["speechiness"] = results["speechiness"]
    song_features["acousticness"] = results["acousticness"]
    song_features["instrumentalness"] = results["instrumentalness"]
    song_features["liveness"] = results["liveness"]
    song_features["valence"] = results["valence"]
    song_features["tempo"] = results["tempo"]
    song_features["time_signature"] = results["time_signature"]
    return song_features


#SIZE OF THE RELEASE?

In [13]:
song = get_song_details("Coldplay","Orphans")
features = get_song_features(song.keys())

{'album': {'album_type': 'single', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/4gzpq5DPGxSnKTe4SA8HAU'}, 'href': 'https://api.spotify.com/v1/artists/4gzpq5DPGxSnKTe4SA8HAU', 'id': '4gzpq5DPGxSnKTe4SA8HAU', 'name': 'Coldplay', 'type': 'artist', 'uri': 'spotify:artist:4gzpq5DPGxSnKTe4SA8HAU'}], 'available_markets': ['AD', 'AE', 'AR', 'AT', 'AU', 'BE', 'BG', 'BH', 'BO', 'BR', 'CA', 'CH', 'CL', 'CO', 'CR', 'CY', 'CZ', 'DE', 'DK', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FR', 'GB', 'GR', 'GT', 'HK', 'HN', 'HU', 'ID', 'IE', 'IL', 'IS', 'IT', 'JO', 'JP', 'KW', 'LB', 'LI', 'LT', 'LU', 'LV', 'MA', 'MC', 'MT', 'MX', 'MY', 'NI', 'NL', 'NO', 'NZ', 'OM', 'PA', 'PE', 'PH', 'PL', 'PS', 'PT', 'PY', 'QA', 'RO', 'SA', 'SE', 'SG', 'SK', 'SV', 'TH', 'TN', 'TR', 'TW', 'US', 'UY', 'VN', 'ZA'], 'external_urls': {'spotify': 'https://open.spotify.com/album/6DX4K0afv5l01Pf6lymJuB'}, 'href': 'https://api.spotify.com/v1/albums/6DX4K0afv5l01Pf6lymJuB', 'id': '6DX4K0afv5l01Pf6lymJuB

In [7]:
song.keys()

dict_keys(['0wJoRiX5K5BxlqZTolB2LD'])

In [8]:
song

{'0wJoRiX5K5BxlqZTolB2LD': {'explicit': False,
  'duration_ms': 170813,
  'disc_number': 1,
  'track_number': 1}}

In [9]:
features

{'danceability': 0.533,
 'energy': 0.905,
 'key': 2,
 'loudness': -5.27,
 'mode': 1,
 'speechiness': 0.0754,
 'acousticness': 0.00876,
 'instrumentalness': 0.578,
 'liveness': 0.0698,
 'valence': 0.486,
 'tempo': 108.9,
 'time_signature': 4}

In [20]:
results = spotify.new_releases(country="US", limit=50, offset=100)

In [21]:
results

{'albums': {'href': 'https://api.spotify.com/v1/browse/new-releases?country=US&offset=100&limit=50',
  'items': [],
  'limit': 50,
  'next': None,
  'offset': 100,
  'previous': 'https://api.spotify.com/v1/browse/new-releases?country=US&offset=50&limit=50',
  'total': 100}}